In [1]:
using Revise

using Statistics
using Symbolics
using ArrayOperations

import Base: getindex,
             @propagate_inbounds

In [5]:
const δ⁻ = Backward{1}(-), Backward{2}(-)
const δ⁺ = Forward{1}(-), Forward{2}(-)

const σ⁻ = Backward{1}(middle), Backward{2}(middle)
const σ⁺ = Forward{1}(middle), Forward{2}(middle);

In [79]:
dims = 1, 1

(1, 1)

In [80]:
# \invw
function grad(ω, γ, a, b, ʍ)
    ⊙(*, ⊙(+, ⊙(*, δ⁻[1](ω), b[1]), ⊙(-, ⊙(*, ⊙(-, σ⁺[1](a[1]), b[1]), γ), σ⁻[1](⊙(*, δ⁺[1](a[1]), γ)))), ʍ[1]),
    ⊙(*, ⊙(+, ⊙(*, δ⁻[2](ω), b[2]), ⊙(-, ⊙(*, ⊙(-, σ⁺[2](a[2]), b[2]), γ), σ⁻[2](⊙(*, δ⁺[2](a[2]), γ)))), ʍ[2])
end

function div(ω, γ, a, b)
    ⊙(+, ⊙(+, ⊙(*, δ⁺[1](ω[1]), a[1]), ⊙(-, ⊙(*, ⊙(-, σ⁻[1](b[1]), a[1]), γ[1]), σ⁺[1](⊙(*, δ⁻[1](b[1]), γ[1])))),
         ⊙(+, ⊙(*, δ⁺[2](ω[2]), a[2]), ⊙(-, ⊙(*, ⊙(-, σ⁻[2](b[2]), a[2]), γ[2]), σ⁺[2](⊙(*, δ⁻[2](b[2]), γ[2])))))
end

div (generic function with 2 methods)

In [81]:
@variables ω[dims...] γ[dims...]
@variables ax[dims...] ay[dims...]; a = (ax, ay)
@variables bx[dims...] by[dims...]; b = (bx, by)
@variables ʍx[dims...] ʍy[dims...]; ʍ = (ʍx, ʍy)

(ʍx[1,1], ʍy[1,1])

In [82]:
qω = grad(ω, γ, a, b, ʍ)
qγ = qω

D = div(qω, qγ, a, b)

(::⊙{typeof(+), Tuple{⊙{typeof(+), Tuple{⊙{typeof(*), Tuple{Ret{Forward{1, typeof(-)}, Tuple{⊙{typeof(*), Tuple{⊙{typeof(+), Tuple{⊙{typeof(*), Tuple{Ret{Backward{1, typeof(-)}, Tuple{Symbolics.Arr{Num, 2}}}, Symbolics.Arr{Num, 2}}}, ⊙{typeof(-), Tuple{⊙{typeof(*), Tuple{⊙{typeof(-), Tuple{Ret{Forward{1, typeof(middle)}, Tuple{Symbolics.Arr{Num, 2}}}, Symbolics.Arr{Num, 2}}}, Symbolics.Arr{Num, 2}}}, Ret{Backward{1, typeof(middle)}, Tuple{⊙{typeof(*), Tuple{Ret{Forward{1, typeof(-)}, Tuple{Symbolics.Arr{Num, 2}}}, Symbolics.Arr{Num, 2}}}}}}}}}, Symbolics.Arr{Num, 2}}}}}, Symbolics.Arr{Num, 2}}}, ⊙{typeof(-), Tuple{⊙{typeof(*), Tuple{⊙{typeof(-), Tuple{Ret{Backward{1, typeof(middle)}, Tuple{Symbolics.Arr{Num, 2}}}, Symbolics.Arr{Num, 2}}}, ⊙{typeof(*), Tuple{⊙{typeof(+), Tuple{⊙{typeof(*), Tuple{Ret{Backward{1, typeof(-)}, Tuple{Symbolics.Arr{Num, 2}}}, Symbolics.Arr{Num, 2}}}, ⊙{typeof(-), Tuple{⊙{typeof(*), Tuple{⊙{typeof(-), Tuple{Ret{Forward{1, typeof(middle)}, Tuple{Symbolics.Arr{N

In [83]:
@syms i::Int j::Int

(i, j)

In [84]:
Dij = D[i, j]

string(Dij)

"(-(1//2)*(-ax[-1 + i, j] + ax[i, j])*γ[-1 + i, j] + ((1//2)*ax[1 + i, j] + (1//2)*ax[i, j] - bx[i, j])*γ[i, j] - (1//2)*(ax[1 + i, j] - ax[i, j])*γ[i, j] + bx[i, j]*(-ω[-1 + i, j] + ω[i, j]))*(-ax[i, j] + (1//2)*bx[-1 + i, j] + (1//2)*bx[i, j])*ʍx[i, j] - (1//2)*(-(1//2" ⋯ 1879 bytes ⋯ " + j])*ay[i, j] - (1//2)*(((1//2)*ay[i, 1 + j] + (1//2)*ay[i, 2 + j] - by[i, 1 + j])*γ[i, 1 + j] - (1//2)*(-ay[i, 1 + j] + ay[i, 2 + j])*γ[i, 1 + j] - (1//2)*(ay[i, 1 + j] - ay[i, j])*γ[i, j] + by[i, 1 + j]*(ω[i, 1 + j] - ω[i, j]))*(by[i, 1 + j] - by[i, j])*ʍy[i, 1 + j]"

In [85]:
Symbolics.get_variables(Dij)

28-element Vector{Any}:
 ax[-1 + i, j]
 ax[i, j]
 γ[-1 + i, j]
 ax[1 + i, j]
 bx[i, j]
 γ[i, j]
 ω[-1 + i, j]
 ω[i, j]
 bx[-1 + i, j]
 ʍx[i, j]
 ax[2 + i, j]
 γ[1 + i, j]
 bx[1 + i, j]
 ⋮
 ay[i, j]
 γ[i, -1 + j]
 ay[i, 1 + j]
 by[i, j]
 ω[i, -1 + j]
 by[i, -1 + j]
 ʍy[i, j]
 ay[i, 2 + j]
 by[i, 1 + j]
 γ[i, 1 + j]
 ω[i, 1 + j]
 ʍy[i, 1 + j]

In [86]:
∂ωij = Differential(ω[i, j])

Differential(ω[i, j])

In [87]:
∂Dij∂ωij = simplify(expand_derivatives(∂ωij(Dij)))

string(∂Dij∂ωij)

"ax[i, j]*(-bx[1 + i, j]*ʍx[1 + i, j] - bx[i, j]*ʍx[i, j]) + (-ax[i, j] + (1//2)*(bx[-1 + i, j] + bx[i, j]))*bx[i, j]*ʍx[i, j] + ay[i, j]*(-by[i, 1 + j]*ʍy[i, 1 + j] - by[i, j]*ʍy[i, j]) + (-ay[i, j] + (1//2)*(by[i, -1 + j] + by[i, j]))*by[i, j]*ʍy[i, j] - (1//2)*(-bx[-1 + i, j] + bx[i, j])*bx[i, j]*ʍx[i, j] + (1//2)*(bx[1 + i, j] - bx[i, j])*bx[1 + i, j]*ʍx[1 + i, j] - (1//2)*(-by[i, -1 + j] + by[i, j])*by[i, j]*ʍy[i, j] + (1//2)*by[i, 1 + j]*(by[i, 1 + j] - by[i, j])*ʍy[i, 1 + j]"